# Descarga de datos de hospitalizados desde Sanidad

## Objetivo

Vamos a hacer una lectura de los datos de hospitalizados publicados en los informes diarios de datos de Covid19 publicados por Sanidad. Este es un [ejemplo](https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Actualizacion_265_COVID-19.pdf) en página 3,  publicado del 4 de Diciembre 2020.
  
Manuel H. Arias  
[@walyt](https://twitter.com/walyt)  

[#escovid19data](https://github.com/montera34/escovid19data)

versión definitiva documentada, para ser publicar en el repo

[@walyt](https://twitter.com/walyt)

## Código

Tenemos un montón de librerias con las que vamos a trabajar, no he tenido ningún problema en instalar aquellas no disponibles en el entorno Anaconda con el que trabajo por medio de `pip install libreria` realizado desde un terminal abierto desde el entorno `env`.

In [1]:
import os.path as pth
import datetime as dt
import time
from glob import glob
import re
import pandas as pd
import numpy as np

import requests
from shutil import copyfile

import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
from matplotlib import cm
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
from matplotlib.dates import (YEARLY, MONTHLY, DateFormatter, WeekdayLocator, MonthLocator,DayLocator,
                              rrulewrapper, RRuleLocator, drange)
import seaborn as sns
import matplotlib.colors as colors

import numpy as np
from datetime import datetime
import seaborn as sns
%matplotlib inline

import fitz
# https://pypi.org/project/PyMuPDF/#description

Preparamos las expresiones regulares que nos ayudarán en la interpretación de la información que sacamos de los pdf.

Definimos variables que nos ayuden en la gestión de los nombres de los ficheros.

In [2]:

fuente_ficheros_pdf_sanidad = '/Users/mharias/documents/proyectos/covid/scrap_sanidad/datos_sanidad/'
dir_csv = '/Users/mharias/documents/proyectos/covid/sanidad_hospital/data/'
URL_reg='https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Actualizacion_{:02d}_COVID-19.pdf'

## Descarga de los ficheros de Sanidad

### Función de para descargar un fichero pdf, copiada del script de [@alfonsotwr](https://github.com/alfonsotwr/snippets/tree/master/covidia-cam)

In [3]:
def descarga(url,num):
    print('Descargando:', url)
    fn=fuente_ficheros_pdf_sanidad+str(num)+'.pdf'
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    with requests.Session() as s:
        r = s.get(url, headers=headers)
    if r.status_code == requests.codes.ok:
        with open(fn, 'wb') as fp:
                fp.write(r.content)
    else:
        print ('Error con el ',num)
    return True

### Descarga de un rango o de un solo pdf

Descargamos el rango completo en el caso de que sea la primera vez. Arrancamos con el 77, pues no pude descifrar el formato de los pdf anteriores.

In [4]:
for i in range(287,297):
    descarga(URL_reg.format(i),i)

Descargando: https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Actualizacion_287_COVID-19.pdf


NameError: name 'fuentes_ficheros_pdf_sanidad' is not defined

### Descarga de un solo fichero

Como ejemplo 266 corresponde al Viernes 4 Diciembre 2020, grabamos el fichero en el directorio local con el nº de orden del documento.

In [5]:
descarga(URL_reg.format(316),316)

Descargando: https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Actualizacion_316_COVID-19.pdf


True

## Parseado y preparación del pandas

In [12]:
columnas = ['fecha','ccaa','Hosp_Totales','Hosp_7días','UCI_Totales','UCI_7dias','Hosp_actual','%CamasCovid',
        'UCI_actual','%CamasUCI','Ingresos_24horas','Altas_24horas']
datos = pd.DataFrame(columns=columnas)

fichero auxiliar que mapea nº de informes con fechas....Mejora: automatizarlo

In [13]:
claves=pd.read_excel(fuente_ficheros_pdf_sanidad+'clave_numero_fecha.xlsx')
claves['Fecha']=claves['Fecha'].astype(str)

<ipython-input-13-bbd0d5524e77>:1: FutureWarning: Your version of xlrd is 1.2.0. In xlrd >= 2.0, only the xls format is supported. As a result, the openpyxl engine will be used if it is installed and the engine argument is not specified. Install openpyxl instead.
  claves=pd.read_excel(fuente_ficheros_pdf_sanidad+'clave_numero_fecha.xlsx')


In [14]:
claves['Fecha'] = claves.apply(lambda x: datetime.strptime(x['Fecha'],format('%Y-%m-%d')).strftime('%Y-%m-%d'),axis=1)

In [15]:
claves[-20:]

,Codigo,Fecha
220,297,2021-01-25
221,298,2021-01-26
222,299,2021-01-27
223,300,2021-01-28
224,301,2021-01-29
225,302,2021-02-01
226,303,2021-02-02
227,304,2021-02-03
228,305,2021-02-04
229,306,2021-02-05


In [16]:
comunidades=['Andalucía','Aragón','Asturias','Baleares','Canarias','Cantabria','Castilla La Mancha',
             'Castilla y León','Cataluña','Ceuta','C. Valenciana','Extremadura','Galicia',
             'Madrid','Melilla','Murcia','Navarra','País Vasco','La Rioja']

## desde 235

In [17]:
rango = np.concatenate([np.arange(235,266),np.arange(267,317)])
opt='text'
for i in rango:
    fn=fuente_ficheros_pdf_sanidad+str(i)+'.pdf'
    doc = fitz.open(fn)
    numero_pagina = 3 if i==282 else 2
    page = doc.load_page(numero_pagina)
    text = page.get_text(opt)
    
    for comunidad in comunidades:
        regexp = r'{} '.format(comunidad)+'((\n\d*[,.]?\d+¥? ){4})'+\
        '(\n \n(\w+[ .\w]*) )((\n\d*[,.]?\d+¥? ))'+\
        '((\n+\d{1,2}[,]?\d*% ))'+\
        '((\n\d*[,.]?\d+¥? ))'+\
        '((\n+\d{1,2}[,]?\d*% ))'+\
        '(((\n\d*[,.]?\d+¥? )){2})'
        pattern = re.compile(regexp)
        c1,c2,c3,c4 = re.search(pattern,text).group(1).replace('\n','').split()
        c5 = re.search(pattern,text).group(6).replace('\n','')
        c6 = re.search(pattern,text).group(8).replace('\n','')
        c7 = re.search(pattern,text).group(9).replace('\n','')
        c8 = re.search(pattern,text).group(11).replace('\n','')
        c9, c10 = re.search(pattern,text).group(13).replace('\n','').split()
        
        #print (i,'--->',comunidad,c1,'-',c2,'-',c3,'-',c4,'-',c5,'-',c6,'-',c7,'-',c8,'-',c9,'-',c10)
        datos = datos.append({'fecha':claves.loc[claves['Codigo']==i,'Fecha'].values[0],
                                                 'ccaa':comunidad,'Hosp_Totales':c1,
                                               'Hosp_7días':c2,'UCI_Totales':c3,
                                                 'UCI_7dias':c4,'Hosp_actual':c5,
                                                 '%CamasCovid':c6,'UCI_actual':c7,
                                                 '%CamasUCI':c8,'Ingresos_24horas':c9,
                                                 'Altas_24horas':c10},ignore_index=True)
        

## antes 235

In [18]:
rango_informes = np.concatenate([np.arange(88,137),np.arange(138,234)])
#rango_informes = np.concatenate([np.arange(88,90)])

for informe in rango_informes:
    fn=fuente_ficheros_pdf_sanidad+str(informe)+'.pdf'
    doc = fitz.open(fn)
    page = doc.load_page(1)
    text = page.get_text(opt)
    
    for comunidad in comunidades:
        regexp = r'{}[*]* '.format(comunidad)+'((\n\d*[,.]?\d+¥? ){6})'
        regexp = r'{}[*]* '.format(comunidad)+'(((\n\d*[,.]?\d+¥? )|(\n- )){6})'
        regexp = r'{}[*¥]* '.format(comunidad)+'(((\n\d*[,.]?\d+¥? )|(\n- )|(\n )){6})'
        cadena = re.search(regexp,text).group(0)  
        #c1,c2,c3,c4,c5,c6 = re.search(regexp,text).group(1).replace('\n','').split()
        c1,c2,c3,c4,c5,c6 = re.search(regexp,text).group(1).split('\n')[1:]
        
        datos = datos.append({'fecha':claves.loc[claves['Codigo']==informe,'Fecha'].values[0],
                                                 'ccaa':comunidad,'Hosp_Totales':c1,
                                               'Hosp_7días':c2,'UCI_Totales':c3,
                                                 'UCI_7dias':c4},ignore_index=True)
        
        
        #print (informe,'--->',comunidad,c1,'-',c2,'-',c3,'-',c4,'-',c5,'-',c6)
        #print (cadena)

In [19]:
datos = datos.sort_values('fecha',ascending=True)

In [20]:
datos

,fecha,ccaa,Hosp_Totales,Hosp_7días,UCI_Totales,UCI_7dias,Hosp_actual,%CamasCovid,UCI_actual,%CamasUCI,Ingresos_24horas,Altas_24horas
1551,2020-04-27,Galicia,2.746,11,83¥,,NaN,NaN,NaN,NaN,NaN,NaN
1557,2020-04-27,La Rioja,1.392,12,85,1,NaN,NaN,NaN,NaN,NaN,NaN
1556,2020-04-27,País Vasco,6.457,31,538,5,NaN,NaN,NaN,NaN,NaN,NaN
1555,2020-04-27,Navarra,1.942,0,130,1,NaN,NaN,NaN,NaN,NaN,NaN
1554,2020-04-27,Murcia,627,0,106,0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1522,2021-02-19,Asturias,7.040,182,520,13,539,"15,43%",127,"38,14%",44,56
1521,2021-02-19,Aragón,12.055,175,1.082,19,503,"11,97%",80,"33,90%",43,60
1520,2021-02-19,Andalucía,36.341,608,3.504,42,2.623,"14,45%",612,"31,60%",209,386
1529,2021-02-19,Ceuta,256,3,30,0,32,"16,00%",7,"41,18%",2,0


In [21]:
datos['fecha'].unique()

array(['2020-04-27', '2020-04-28', '2020-04-29', '2020-04-30',
       '2020-05-01', '2020-05-02', '2020-05-03', '2020-05-04',
       '2020-05-05', '2020-05-06', '2020-05-07', '2020-05-08',
       '2020-05-09', '2020-05-10', '2020-05-11', '2020-05-12',
       '2020-05-13', '2020-05-14', '2020-05-15', '2020-05-16',
       '2020-05-17', '2020-05-18', '2020-05-19', '2020-05-20',
       '2020-05-21', '2020-05-22', '2020-05-23', '2020-05-24',
       '2020-05-25', '2020-05-26', '2020-05-27', '2020-05-28',
       '2020-05-29', '2020-05-30', '2020-05-31', '2020-06-01',
       '2020-06-02', '2020-06-03', '2020-06-04', '2020-06-05',
       '2020-06-06', '2020-06-07', '2020-06-08', '2020-06-09',
       '2020-06-10', '2020-06-11', '2020-06-12', '2020-06-13',
       '2020-06-14', '2020-06-16', '2020-06-17', '2020-06-18',
       '2020-06-19', '2020-06-20', '2020-06-21', '2020-06-22',
       '2020-06-23', '2020-06-24', '2020-06-25', '2020-06-26',
       '2020-06-27', '2020-06-28', '2020-06-29', '2020-

In [22]:
datos.to_csv(dir_csv+'datos_sanidad_hosp.csv',index=False,index_label=False)

## Parseado pre 235, con la página 2, Tabla 2

In [ ]:
i = 87
fn=fuente_ficheros_pdf_sanidad+str(i)+'.pdf'
doc = fitz.open(fn)
page = doc.load_page(1)
text = page.get_text(opt)
comunidad='Galicia'
regexp = r'{}[*]? '.format(comunidad)+'((\n\d*[,.]?\d+¥? ){6})'
regexp = r'{}[*¥]* '.format(comunidad)+'(((\n\d*[,.]?\d+¥? )|(\n- )|(\n )){6})'
cadena = re.search(regexp,text).group(1).split(('\n'))
print (cadena)


In [ ]:
text

In [ ]:
i = 211
fn=fuente_ficheros_pdf_sanidad+str(i)+'.pdf'
doc = fitz.open(fn)
page = doc.load_page(1)
text = page.get_text(opt)
comunidad='Andalucía'
for comunidad in comunidades:
    regexp = r'{}[*]* '.format(comunidad)+'((\n\d*[,.]?\d+¥? ){6})'
    regexp = r'{}[*]* '.format(comunidad)+'(((\n\d*[,.]?\d+¥? )|(\n- )){6})'
    regexp = r'{}[*¥]* '.format(comunidad)+'(((\n\d*[,.]?\d+¥? )|(\n- )|(\n )){6})'
    cadena = re.search(regexp,text).group(0)
    c1,c2,c3,c4,c5,c6 = re.search(regexp,text).group(1).split('\n')[1:]
    print (cadena)
    print (informe,'--->',comunidad,c1,'-',c2,'-',c3,'-',c4,'-',c5,'-',c6)

In [ ]:
text

In [ ]:
rango_informes = np.concatenate([np.arange(88,137),np.arange(138,234)])
#rango_informes = np.concatenate([np.arange(88,90)])

for informe in rango_informes:
    fn=fuente_ficheros_pdf_sanidad+str(informe)+'.pdf'
    doc = fitz.open(fn)
    page = doc.load_page(1)
    text = page.get_text(opt)
    
    for comunidad in comunidades:
        regexp = r'{}[*]* '.format(comunidad)+'((\n\d*[,.]?\d+¥? ){6})'
        regexp = r'{}[*]* '.format(comunidad)+'(((\n\d*[,.]?\d+¥? )|(\n- )){6})'
        regexp = r'{}[*¥]* '.format(comunidad)+'(((\n\d*[,.]?\d+¥? )|(\n- )|(\n )){6})'
        cadena = re.search(regexp,text).group(0)  
        #c1,c2,c3,c4,c5,c6 = re.search(regexp,text).group(1).replace('\n','').split()
        c1,c2,c3,c4,c5,c6 = re.search(regexp,text).group(1).split('\n')[1:]
        
        datos = datos.append({'fecha':claves.loc[claves['Codigo']==informe,'Fecha'].values[0],
                                                 'ccaa':comunidad,'Hosp_Totales':c1,
                                               'Hosp_7días':c2,'UCI_Totales':c3,
                                                 'UCI_7dias':c4},ignore_index=True)
        
        
        print (informe,'--->',comunidad,c1,'-',c2,'-',c3,'-',c4,'-',c5,'-',c6)
        #print (cadena)

In [ ]:
re.search(regexp,text).group(1)  

In [ ]:
datos=datos.sort_values(['fecha','ccaa']).reset_index()

In [ ]:
datos.to_csv(dir_csv+'datos_sanidad_hosp.csv',index=False,index_label=False)

In [ ]:
datos

## Parseado pre 235, con la página 3 Tabla 3, % camas, ingresados diarios 

In [ ]:
i = 211
fn=fuente_ficheros_pdf_sanidad+str(i)+'.pdf'
doc = fitz.open(fn)
page = doc.load_page(3)
text = page.get_text(opt)
comunidad='Andalucía'
regexp = r'{}[*¥]* '.format(comunidad)+'((\n\d*[,.]?\d+¥? ))'+'((\n+\d{1,2}[,]?\d*% ))'+\
'((\n\d*[,.]?\d+¥? ))'+'((\n+\d{1,2}[,]?\d*% ))'+'((\n\d*[,.]?\d+¥? )){2}'
#para el 211
regexp = r'{}[*¥]* '.format(comunidad)+'((\n\d*[,.]?\d+¥? )){2}'+'((\n+\d{1,2}[,]?\d*% ))'+'((\n\d*[,.]?\d+¥? )){2}'

cadena = re.search(regexp,text).group(0).split(('\n'))[1:]
print (cadena)


In [ ]:
text

## un informe todas las comunidades

In [ ]:
i = 233
fn=fuente_ficheros_pdf_sanidad+str(i)+'.pdf'
doc = fitz.open(fn)

page = doc.load_page(2)
text = page.get_text(opt)
comunidad='Andalucía'
for comunidad in comunidades:
    regexp = r'{}[*¥]* '.format(comunidad)+'((\n\d*[,.]?\d+¥? ))'+'((\n+\d{1,2}[,]?\d*% ))'+\
    '((\n\d*[,.]?\d+¥? ))'+'((\n+\d{1,2}[,]?\d*% ))'+'((\n\d*[,.]?\d+¥? )){2}'
    cadena = re.search(regexp,text).group(0)
    c1,c2,c3,c4,c5,c6 = re.search(regexp,text).group(0).split('\n')[1:]
    print (cadena)
    print (informe,'--->',comunidad,c1,'-',c2,'-',c3,'-',c4,'-',c5,'-',c6)

## todos los informes todas las comunidades

In [ ]:
#rango_informes = np.concatenate([np224.arange(88,137),np.arange(138,234)])
rango_informes = np.concatenate([np.arange(212,215)])

for informe in rango_informes:
    fn=fuente_ficheros_pdf_sanidad+str(informe)+'.pdf'
    doc = fitz.open(fn)
    n_pagina = 3 if informe<=224 else 2
    page = doc.load_page(n_pagina)
    text = page.get_text(opt)
    
    for comunidad in comunidades:
        regexp = r'{}[*¥]* '.format(comunidad)+'((\n\d*[,.]?\d+¥? ))'+'((\n+\d{1,2}[,]?\d*% ))'+\
    '((\n\d*[,.]?\d+¥? ))'+'((\n+\d{1,2}[,]?\d*% ))'+'((\n\d*[,.]?\d+¥? )){2}'
        cadena = re.search(regexp,text).group(0)
        c1,c2,c3,c4,c5,c6 = re.search(regexp,text).group(0).split('\n')[1:]
        #print (cadena)
        print (informe,'--->',comunidad,c1,'-',c2,'-',c3,'-',c4,'-',c5,'-',c6)
        
        

## 211 y anteriores, no publican % camas hospital

In [ ]:
#rango_informes = np.concatenate([np224.arange(88,137),np.arange(138,234)])
rango_informes = np.concatenate([np.arange(204,212)])

for informe in rango_informes:
    fn=fuente_ficheros_pdf_sanidad+str(informe)+'.pdf'
    doc = fitz.open(fn)
    n_pagina = 3
    page = doc.load_page(n_pagina)
    text = page.get_text(opt)
    
    for comunidad in comunidades:
        regexp = r'{}[*¥]* '.format(comunidad)+'((\n\d*[,.]?\d+¥? )){2}'+'((\n+\d{1,2}[,]?\d*% ))'+'((\n\d*[,.]?\d+¥? )){2}'
        cadena = re.search(regexp,text).group(0)
        c1,c2,c3,c4,c5 = re.search(regexp,text).group(0).split('\n')[1:]
        #print (cadena)
        print (informe,'--->',comunidad,c1,'-',c2,'-',c3,'-',c4,'-',c5,'-',c6)
        
        

In [ ]:
text